In [1]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from raw_yfinance_ingestion import RawYFIngestion  
from lab_table_manager import TableManager
import yfinance as yf

In [ ]:
def merge_iceberg_into_pg(spark, iceberg_source_table, conn_config_file, pg_database_name, pg_sink_table, pg_truncate_script, pg_merge_script):   
    try:             
        # Get finalytics connetion info
        conn_config_file='cfg_connections.yaml'
        finalytics=PgDBManager(conn_config_file, 'finalytics')
        pg_url=finalytics.jdbc_url
        pg_driver=finalytics.driver
        
        df_source=spark.read.table(iceberg_source_table)          
        finalytics.execute_sql_script(pg_truncate_script)
        
        # Write DataFrame to PostgreSQL
        df_source.write.jdbc(url=pg_url, table=pg_sink_table, mode="append", properties={"driver": pg_driver})        
        finalytics.execute_sql_script(pg_merge_script)
    except Exception as e:
        print(f"Error loading pg finalytics: {e}")        